In [1]:
# Imports 
import numpy as np
import random
from datetime import datetime

In [2]:
# Parameters
n_cities = 20

n_population = 100

mutation_rate = 0.3

In [3]:
# Generating a list of coordenades representing each city
coordinates_list = [[x,y] for x,y in zip(np.random.randint(0,100,n_cities),np.random.randint(0,100,n_cities))]
names_list = np.array(['Berlin', 'London', 'Moscow', 'Barcelona', 'Rome', 'Paris', 'Vienna', 'Munich', 'Istanbul', 'Kyiv', 'Bucharest', 'Minsk', 'Warsaw', 'Budapest', 'Milan', 'Prague', 'Sofia', 'Birmingham', 'Brussels', 'Amsterdam'])
cities_dict = { x:y for x,y in zip(names_list,coordinates_list)}

# Function to compute the distance between two points
def compute_city_distance_coordinates(a,b):
    return ((a[0]-b[0])**2+(a[1]-b[1])**2)**0.5

def compute_city_distance_names(city_a, city_b, cities_dict):
    return compute_city_distance_coordinates(cities_dict[city_a], cities_dict[city_b])

cities_dict

{'Berlin': [12, 54],
 'London': [90, 49],
 'Moscow': [92, 96],
 'Barcelona': [45, 30],
 'Rome': [21, 47],
 'Paris': [10, 22],
 'Vienna': [81, 21],
 'Munich': [47, 59],
 'Istanbul': [97, 52],
 'Kyiv': [72, 36],
 'Bucharest': [54, 10],
 'Minsk': [28, 3],
 'Warsaw': [28, 18],
 'Budapest': [45, 27],
 'Milan': [1, 24],
 'Prague': [76, 33],
 'Sofia': [53, 68],
 'Birmingham': [17, 44],
 'Brussels': [95, 63],
 'Amsterdam': [39, 34]}

In [4]:
# First step: Create the first population set
def genesis(city_list, n_population):

    population_set = []
    for i in range(n_population):
        #Randomly generating a new solution
        sol_i = city_list[np.random.choice(list(range(n_cities)), n_cities, replace=False)]
        population_set.append(sol_i)
    return np.array(population_set)

population_set = genesis(names_list, n_population)
population_set

array([['Birmingham', 'Minsk', 'Budapest', ..., 'Barcelona', 'Brussels',
        'Warsaw'],
       ['Barcelona', 'Moscow', 'Amsterdam', ..., 'Kyiv', 'Prague',
        'Brussels'],
       ['Milan', 'Barcelona', 'Bucharest', ..., 'Berlin', 'Amsterdam',
        'Paris'],
       ...,
       ['Kyiv', 'Budapest', 'Vienna', ..., 'Amsterdam', 'Paris',
        'Warsaw'],
       ['Milan', 'Warsaw', 'Barcelona', ..., 'Prague', 'Brussels',
        'Berlin'],
       ['Moscow', 'Munich', 'Sofia', ..., 'Milan', 'Bucharest', 'Paris']],
      dtype='<U10')

In [5]:
def fitness_eval(city_list, cities_dict):
    total = 0
    for i in range(n_cities-1):
        a = city_list[i]
        b = city_list[i+1]
        total += compute_city_distance_names(a,b, cities_dict)
    return total

In [6]:
def get_all_fitnes(population_set, cities_dict):
    fitnes_list = np.zeros(n_population)

    #Looping over all solutions computing the fitness for each solution
    for i in  range(n_population):
        fitnes_list[i] = fitness_eval(population_set[i], cities_dict)

    return fitnes_list

fitnes_list = get_all_fitnes(population_set,cities_dict)
fitnes_list

array([ 930.58571548,  885.85251657,  960.16405022,  857.00935094,
        926.9414824 , 1048.58176462,  732.99051332,  871.26473914,
       1047.48284566,  966.19126693,  917.79491175,  851.25621289,
        714.11533788,  856.36429593,  882.64989805,  959.00541606,
        944.55039602, 1095.84691672,  915.64665525,  859.62824722,
        709.22645604,  925.36099491,  888.58437474,  840.93149388,
       1033.09660739,  833.49784126,  829.3302081 ,  819.58750203,
        962.68463712, 1087.38523921,  902.11415236, 1092.38873216,
        962.76755719,  953.91952401,  828.74509797,  799.6638612 ,
        873.17870944, 1017.77011789,  835.82362927,  864.2959544 ,
        986.12132331,  919.95092228,  863.67895647, 1033.80652467,
       1065.20672104,  963.65486611, 1084.50047516,  957.19944697,
        857.38269586,  933.67837027,  967.42952989, 1044.9932031 ,
        831.26828081,  901.75938206,  840.13121461,  792.63056974,
       1045.55434523,  995.90421607,  752.17198798, 1060.45005

In [7]:
def progenitor_selection(population_set,fitnes_list):
    total_fit = fitnes_list.sum()
    prob_list = (total_fit/fitnes_list)
    prob_list = prob_list/prob_list.sum()
    
    #Notice there is the chance that a progenitor. mates with oneself
    progenitor_list_a = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    progenitor_list_b = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    
    progenitor_list_a = population_set[progenitor_list_a]
    progenitor_list_b = population_set[progenitor_list_b]
    
    
    return np.array([progenitor_list_a,progenitor_list_b])


progenitor_list = progenitor_selection(population_set,fitnes_list)
progenitor_list[0][2]

array(['Minsk', 'Paris', 'Prague', 'Amsterdam', 'Kyiv', 'Rome', 'Vienna',
       'Munich', 'Istanbul', 'London', 'Bucharest', 'Sofia', 'Berlin',
       'Birmingham', 'Warsaw', 'Moscow', 'Milan', 'Budapest', 'Brussels',
       'Barcelona'], dtype='<U10')

In [8]:
def mate_progenitors(prog_a, prog_b):
    offspring = prog_a[0:5]

    for city in prog_b:

        if not city in offspring:
            offspring = np.concatenate((offspring,[city]))

    return offspring
            
    
    
def mate_population(progenitor_list):
    new_population_set = []
    for i in range(progenitor_list.shape[1]):
        prog_a, prog_b = progenitor_list[0][i], progenitor_list[1][i]
        offspring = mate_progenitors(prog_a, prog_b)
        new_population_set.append(offspring)
        
    return new_population_set

new_population_set = mate_population(progenitor_list)
new_population_set[0]

array(['Munich', 'Istanbul', 'Milan', 'Berlin', 'Minsk', 'Amsterdam',
       'Paris', 'Sofia', 'Rome', 'Budapest', 'Warsaw', 'Prague',
       'Barcelona', 'London', 'Vienna', 'Moscow', 'Kyiv', 'Brussels',
       'Bucharest', 'Birmingham'], dtype='<U10')

In [9]:
def mutate_offspring(offspring):
    for q in range(int(n_cities*mutation_rate)):
        a = np.random.randint(0,n_cities)
        b = np.random.randint(0,n_cities)

        offspring[a], offspring[b] = offspring[b], offspring[a]

    return offspring
    
    
def mutate_population(new_population_set):
    mutated_pop = []
    for offspring in new_population_set:
        mutated_pop.append(mutate_offspring(offspring))
    return mutated_pop

mutated_pop = mutate_population(new_population_set)
mutated_pop[0]

array(['Munich', 'Kyiv', 'Birmingham', 'Bucharest', 'Minsk', 'Brussels',
       'Paris', 'Sofia', 'Milan', 'Budapest', 'Warsaw', 'Prague',
       'Moscow', 'London', 'Vienna', 'Barcelona', 'Istanbul', 'Amsterdam',
       'Berlin', 'Rome'], dtype='<U10')

In [10]:
best_solution = [-1,np.inf,np.array([])]
for i in range(10000):
    if i%50==0: print(i, best_solution[1], fitnes_list.mean(), datetime.now().strftime("%d/%m/%y %H:%M"))
    fitnes_list = get_all_fitnes(mutated_pop,cities_dict)
    
    #Saving the best solution
    if fitnes_list.min() < best_solution[1]:
        best_solution[0] = i
        best_solution[1] = fitnes_list.min()
        best_solution[2] = np.array(mutated_pop)[fitnes_list.min() == fitnes_list]
    
    progenitor_list = progenitor_selection(population_set,fitnes_list)
    new_population_set = mate_population(progenitor_list)
    
    mutated_pop = mutate_population(new_population_set)

0 inf 915.5517227877122 03/10/23 12:41
50 557.4420796446036 906.6536955279906 03/10/23 12:41
100 557.4420796446036 918.6191098830853 03/10/23 12:41
150 557.4420796446036 914.5113616674621 03/10/23 12:41
200 557.4420796446036 900.1021999579935 03/10/23 12:41
250 557.4420796446036 916.3669050814119 03/10/23 12:41
300 557.4420796446036 917.4154260853677 03/10/23 12:41
350 557.4420796446036 905.2247697108351 03/10/23 12:41
400 557.4420796446036 913.6460571237928 03/10/23 12:41
450 557.4420796446036 927.9187954132406 03/10/23 12:41
500 557.4420796446036 921.4591768986658 03/10/23 12:41
550 557.4420796446036 929.2508799778784 03/10/23 12:42
600 557.4420796446036 922.4744980515494 03/10/23 12:42
650 557.4420796446036 901.5685724636018 03/10/23 12:42
700 533.2731530564902 898.4875405976064 03/10/23 12:42
750 533.2731530564902 913.7367187229409 03/10/23 12:42
800 533.2731530564902 907.8688847706256 03/10/23 12:42
850 533.2731530564902 910.5551761445273 03/10/23 12:42
900 533.2731530564902 917.5

In [11]:
best_solution

[3278,
 524.2065420262528,
 array([['Moscow', 'Brussels', 'Warsaw', 'Bucharest', 'Kyiv', 'Vienna',
         'Prague', 'Istanbul', 'London', 'Barcelona', 'Budapest', 'Sofia',
         'Munich', 'Rome', 'Amsterdam', 'Minsk', 'Berlin', 'Birmingham',
         'Paris', 'Milan']], dtype='<U10')]